# Computational Theory: Analysis of the Countdown Numbers Game

*Jakub Prochnicki G00373793*

---

# Table of Contents
1. [Introduction](#intro)
2. [Explanation of the Countdown Numbers Game](#concepts)
3. [Analysis of the Computational Complexity](#oracle)
4. [Approach to Solving the Game](#implementation)
5. [Conclusion](#conclusion)

# Introduction
---

This notebook constitutes my assessment submission for the Computational Theory Module at ATU Galway. The aim of this notebook is to provide an analysis of the Countdown Numbers Game. Firstly, I will provide an explanation of the game and give an example of the game in action. Following the explanation, I delve into an analysis of the computational complexity of the game and provide various approaches to solving the game, namely: Brute Force Approach, Heuristic Approach and Reverse Polish Notation. Lastly, I will evaluate my findings from solving the problem using various approaches

# Explanation of the Countdown Numbers Game
---
The Countdown Numbers Game allows people to showcase their math skills on a British aired TV Show, Countdown. The aim of the game is to reach a specific target number using a set of six other numbers and basic arithmetic operations. In order to better understand the game, I have come up with the following breakdown of how a round of Countdown works.

- The player selects six numbers from two groups : The first group consisting of four large numbers: 25, 50, 75, and 100. The other group holds twenty smaller numbers ranging from 1 to 10, with each number appearing twice. The player has the freedom to use any combination of numbers from these two groups.

- The game generates a target number, a three digit number.
- The player is then given a 30 second window and is given the task of using a combination of arithmetic operations in order to come as close as possible to the target number. Each of the six numbers can be used once and players are not obliged to use all six numbers. The operations must however result in a whole number at every step.
- The score is determined by the closeness of the player's number to the target score. A perfect score of 10 is awarded to players where they have reached the target number.

  *need to give example of a game in action*

# Analysing the Computational Complexity of the Game
---
In order to analyse the computational complexity, it is necessary to consder how difficult it is to find a solution to reach the target number using a set of numbers and arithmetic operations. This analysis typically focuses on the theoretical limits of solving this problem using algorithms, and the complexity can be described in terms of both time and space (memory).

# References 
--- 
[1] Lynch, J. and Yan Weng. “The Computational Complexity of Finding Arithmetic Expressions With and Without Parentheses.” https://www.semanticscholar.org/reader/21f455ca0bbf9a355611bc0593dd1cf8a8d32584

“Countdown” numbers game: the lowdown https://www.daitx.com/2016/05/01/countdown-math/
